In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import spacy
from transformers import CamembertModel, CamembertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Récupération du dataset et filtres de données

In [2]:
# Données à récupérer via le fichier movies.csv
path = "Documents/Cours/Cours Simplon/Projet fil rouge - développer un moteur de recommandation de film/movies.csv"
data = pd.read_csv("movies.csv")
print(data.shape)

(8019, 12)


In [3]:
# Création d'un dataframe
df = pd.DataFrame(data)

In [4]:
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

## Chargement du modele BERT

In [5]:
# Charger le tokenizer et le modèle CamemBERT pré-entraîné
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model_camembert = CamembertModel.from_pretrained('camembert-base')

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

c:\Users\Leo\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Leo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

## Pretraitement des données

In [6]:
# Prétraitement des données textuelles avec spaCy
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [7]:
# Appliquer le prétraitement aux descriptions des films
data['processed_description'] = data['desc_fr'].apply(preprocess_text)

In [8]:
# Embedding des descriptions de films avec CamemBERT
embeddings = []

for desc in data['processed_description']:
    inputs = tokenizer(desc, return_tensors="pt")
    outputs = model_camembert(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

## Modele BERT

In [9]:
# Fonction pour obtenir les recommandations de films avec CamemBERT
def get_recommendations_camembert(movie_title, n=5):
    # Trouver l'indice du film dans le dataframe
    movie_index = data[data['titre'] == movie_title].index[0]

    # Obtenir l'embedding CamemBERT du film
    movie_embedding = embeddings[movie_index]

    # Calcul de la similarité cosinus avec toutes les autres descriptions
    similarities = cosine_similarity(movie_embedding.reshape(1, -1), embeddings).flatten()

    # Trier les indices par ordre décroissant de similarité
    related_movies_indices = similarities.argsort()[::-1]

    # Exclure le film lui-même de la liste des recommandations
    related_movies_indices = [i for i in related_movies_indices if i != movie_index]

    # Récupérer les titres des films recommandés
    recommendations = data['titre'].iloc[related_movies_indices][:n].tolist()

    return recommendations

In [10]:
# Exemple d'utilisation avec CamemBERT
movie_title = 'Oppenheimer'
recommendations_camembert = get_recommendations_camembert(movie_title)
print(f"Recommandations pour '{movie_title}' avec CamemBERT:")
for i, title in enumerate(recommendations_camembert):
    print(f"{i+1}. {title}")

Recommandations pour 'Oppenheimer' avec CamemBERT:
1. Le quatrième protocole
2. Sunshine
3. Le prodige
4. In the Loop
5. Good Night, and Good Luck


In [11]:
# Enregistrement du modèle CamemBERT
model_camembert.save_pretrained('save_pretrained')

# Enregistrement du tokenizer CamemBERT
tokenizer.save_pretrained('save_pretrained')

('save_pretrained\\tokenizer_config.json',
 'save_pretrained\\special_tokens_map.json',
 'save_pretrained\\sentencepiece.bpe.model',
 'save_pretrained\\added_tokens.json')